# Model H

This is a script showing results of **model H** in the ablation studies. The model is built using the following configuration:

- **Backbone:** Transformer
- **Graph type:** Heterogeneous
- **Loss function:** Bayesian Personalized Ranking Loss
- **Embeddings used:** average
- **Keyword popularity used:** No

*This model introduces a heterogeneous version of the graph transformer backbone.*

## **Setting up environment**

---



### Loading libraries

In [1]:
import os
import sys

sys.path.insert(0, os.path.abspath(".."))

import torch
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from typing import Optional

from torch import Tensor
from torch.nn import ModuleList
import torch.nn.functional as F
import torch.nn.functional as F
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LRScheduler
from torch_geometric.nn import Linear
from torch_geometric.typing import Adj, OptTensor
from torch_geometric.nn.conv import TransformerConv
from torch_geometric.data import Data

from util.torch_geometric import get_results
from util.heterogeneous.dataset import (DatasetEuCoHT, load_dataset)
from util.heterogeneous.model import ModelEuCoHT
from util.heterogeneous.train import (
    train,
    test,
    evaluate
)

### Global variables

In [3]:
# Model name
model_name = 'H'
# Dataset save filepath
dataset_save_filepath_prefix = '../data/dataset_heterogeneous'
# Device name
device = 'cpu'

# Model configuration
model_config = dict(
    hidden_channels=64,
    learning_rate=1e-3,
    num_layers=4,
    num_heads=2,
    num_epochs=50,
    num_recommendations=10,
    include_linear_layers=True,
    include_activation_layers=True,
    num_bootstrap=10
)
target_edge_type = ('author', 'co_authors', 'author')
target_node_type = 'author'

# Set seaborn theme
sns.set_theme(style="whitegrid", palette="pastel")

## Model training


---



### Model definition

In [4]:
class ModelEuCoH(ModelEuCoHT):
    def __init__(self,
                 input_channels: int,
                 hidden_channels: int,
                 num_layers: int,
                 num_recommendations: int,
                 num_heads: int,
                 author_node_id_map: dict,
                 author_id_map: dict,
                 include_linear_layers: bool,
                 include_activation_layers: bool):
        super().__init__(
            hidden_channels=hidden_channels,
            num_layers=num_layers,
            num_recommendations=num_recommendations,
            author_node_id_map=author_node_id_map,
            author_id_map=author_id_map,
            include_linear_layers=include_linear_layers,
            include_activation_layers=include_activation_layers
        )
        # Number of attention heads
        self.num_heads = num_heads
        
        # # Initialize the convolutional layers
        self.conv_layers = ModuleList([
            TransformerConv(
                in_channels=(-1, -1), 
                out_channels=hidden_channels,
                heads=self.num_heads,
                dropout=0.2
            )
            for i in range(self.num_layers)
        ])

        # Linear layers
        self.lin_layers = ModuleList([
            Linear(-1, hidden_channels * self.num_heads) for i in range(self.num_layers)
        ])

        # Batch norm layers
        self.bn_layers = ModuleList([
            torch.nn.BatchNorm1d(hidden_channels * self.num_heads) for i in range(self.num_layers)
        ])

    

### Training the model

In [5]:
def load_model(data: Data, model_config: dict, author_node_id_map: dict, author_id_map: dict):
    # Initialize the model
    model = ModelEuCoH(
        input_channels=data.num_features,
        hidden_channels=model_config['hidden_channels'],
        num_recommendations=model_config['num_recommendations'],
        num_layers=model_config['num_layers'],
        num_heads=model_config['num_heads'],
        author_node_id_map=author_node_id_map,
        author_id_map=author_id_map,
        include_linear_layers=model_config['include_linear_layers'],
        include_activation_layers=model_config['include_activation_layers']
    ).to(device)
    
    # Initialize the optimizer
    optimizer: Optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=model_config['learning_rate']
    )
    
    # Initialize the scheduler
    scheduler: LRScheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer=optimizer,
        mode='min',
        factor=0.5,
        patience=10
    )
    
    return model, data, optimizer, scheduler

In [ ]:
results: list = list()
for bootstrap_id in range(model_config['num_bootstrap']):
    print(f'Processing bootstrap dataset ix={bootstrap_id}...')
    dataset_filepath = f'{dataset_save_filepath_prefix}_b{bootstrap_id}.pkl'
    data, author_id_map, author_node_id_map = load_dataset(dataset_filepath=dataset_filepath, device=device)
    model, data, optimizer, scheduler = load_model(data, model_config, author_node_id_map, author_id_map)
    for epoch in range(1, model_config['num_epochs'] + 1):
        # ------ Train
        train_loss: float = train(
            model=model,
            data=data,
            optimizer=optimizer,
            target_edge_type=target_edge_type,
            target_node_type=target_node_type
        )
        # ------ Test
        test_loss: float = test(
            model=model,
            data=data,
            target_edge_type=target_edge_type,
        )
        scheduler.step(test_loss)
        # ------ Evaluate
        evaluation_results: dict = evaluate(
            num_recommendations=model_config['num_recommendations'],
            model=model,
            data=data,
            target_edge_type=target_edge_type,
            target_node_type=target_node_type
        )
    
        # Save results
        epoch_result = get_results(
            epoch=epoch,
            train_loss=train_loss,
            test_loss=test_loss,
            evaluation_results=evaluation_results,
            bootstrap_id=bootstrap_id,
            log_every_n_epochs=10
        )
        results.append(epoch_result)

Processing bootstrap dataset ix=0...


/home/eucollab/eutopia-colllaboration/ecr-recommender/.venv/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.loa

Epoch 10, train loss: 0.0116, test loss: 3.8603, precision@k: 0.0051, recall@k: 0.0159, MAP@k: 0.0055, MRR@k: 0.0128, NDCG@k: 0.0101, HitRate@k: 0.0379


### Model evaluation

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv(f'../results/results_Model_bootstrapped_{model_name}.csv', index=False)

In [ ]:
# Generate loss curve
# Train loss
sns.lineplot(data=results_df, x='Epoch', y='Train Loss')
# Test loss
sns.lineplot(data=results_df, x='Epoch', y='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss curve')
plt.yscale('log')
plt.show()

In [ ]:
# Generate evaluation metrics plot
sns.lineplot(data=results_df, x='Epoch', y='Precision@k')
sns.lineplot(data=results_df, x='Epoch', y='Recall@k')
sns.lineplot(data=results_df, x='Epoch', y='MAP@k')
sns.lineplot(data=results_df, x='Epoch', y='MRR@k')
sns.lineplot(data=results_df, x='Epoch', y='NDCG@k')
sns.lineplot(data=results_df, x='Epoch', y='HitRate@k')
plt.xlabel('Epoch')
plt.ylabel('Performance')
plt.title('Performance metrics')
plt.show()